<a href="https://colab.research.google.com/github/konmavedant/Docker/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!free -h  # Check RAM
!nvidia-smi  # Check GPU (if available)

               total        used        free      shared  buff/cache   available
Mem:            12Gi       787Mi       7.5Gi       1.0Mi       4.4Gi        11Gi
Swap:             0B          0B          0B
Fri Feb 28 18:13:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8        

In [ ]:
!pip install transformers torch datasets accelerate bitsandbytes
!apt-get update && apt-get install -y git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Step 1: Import Libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Step 2: Check Resources
print("Checking available resources:")
!free -h  # Display RAM
!nvidia-smi  # Display GPU info (T4 should show ~16GB VRAM)

# Step 3: Define Model and Quantization Settings
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"  # Replace with latest distilled model from Hugging Face
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization to fit T4 VRAM
    bnb_4bit_compute_dtype=torch.float16  # Optimize for T4 GPU
)

# Step 4: Load Model and Tokenizer
print(f"Loading {model_name} with 4-bit quantization onto T4 GPU...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="cuda:0",  # Explicitly use T4 GPU
        torch_dtype=torch.float16  # Use FP16 for faster GPU inference
    )
    print("Model loaded successfully onto GPU!")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Step 5: Maximize Context Length
initial_context_length = 16384  # Start with 16K tokens, T4 can handle more with RAM support
tokenizer.model_max_length = initial_context_length
print(f"Setting initial context length to {initial_context_length} tokens.")

# Function to test and adjust context length
def test_context_length(max_length, test_text):
    tokenizer.model_max_length = max_length
    try:
        inputs = tokenizer(test_text, return_tensors="pt", truncation=True, max_length=max_length).to("cuda:0")
        outputs = model.generate(**inputs, max_new_tokens=10)
        print(f"Success at {max_length} tokens")
        return True
    except RuntimeError as e:
        print(f"Failed at {max_length} tokens: {e}")
        return False

# Generate a long test text (~10K tokens worth)
test_text = "AI is advancing rapidly in 2025, transforming industries and research. " * 5000

# Dynamically adjust context length
context_lengths = [8192, 16384, 32768]  # Test these lengths, T4 + 12.7GB RAM can push higher
max_working_length = 8192  # Default safe value
for length in context_lengths:
    if test_context_length(length, test_text):
        max_working_length = length
    else:
        break

tokenizer.model_max_length = max_working_length
print(f"Set maximum working context length to {max_working_length} tokens.")

# Step 6: Run Inference
def run_inference(prompt):
    print(f"Running inference with prompt: {prompt[:50]}...")
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_working_length).to("cuda:0")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # Avoid padding warnings
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated output:")
    print(result)
    print("\nResource usage after inference:")
    !free -h
    !nvidia-smi

# Test with a sample prompt
sample_prompt = "Summarize AI advancements in 2025 based on current trends."
run_inference(sample_prompt)

# Step 7: Test Long Context
long_prompt = "Repeat this sentence to test context: AI is the future. " * 1000
print(f"Testing long context with ~{len(long_prompt.split())} words...")
run_inference(long_prompt)

# Step 8: Cleanup (Optional)
print("Cleaning up to free GPU and RAM...")
del model
del tokenizer
torch.cuda.empty_cache()
print("Cleanup complete. Check resources:")
!free -h
!nvidia-smi

Checking available resources:
               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.5Gi       330Mi       1.0Mi        10Gi        10Gi
Swap:             0B          0B          0B
Fri Feb 28 18:18:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                   

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

...

Success at 8192 tokens


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Success at 16384 tokens


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Success at 32768 tokens
Set maximum working context length to 32768 tokens.
Running inference with prompt: Summarize AI advancements in 2025 based on current...
Generated output:
Summarize AI advancements in 2025 based on current trends. What specific areas are AI excelling in? What are the challenges it faces?

AI is making remarkable strides in 2025, driven by advancements in machine learning and neural networks. Specific areas where AI excels include natural language processing, computer vision, robotics, and autonomous systems. These areas leverage cutting-edge algorithms and hardware improvements, leading to breakthroughs in tasks like translation, image recognition, robotic manipulation, and self-driving cars. Challenges AI faces include ethical concerns, data biases, computational costs

Resource usage after inference:
               total        used        free      shared  buff/cache   available
Mem:            12Gi       3.1Gi       250Mi        15Mi       9.4Gi       9.3Gi
